In [1]:
""" This File is our baseline tool for getting the Open, High, Low, Close, Volume for BTC.
    It uses Binance API and creates DataFrames for further data manipulation and TA.
    We will use this for the Logistic Regression."""
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import mplfinance as mpl
import os
import numpy as np
from hvplot import hvPlot
import pandas as pd
from ta import add_all_ta_features
from ta.utils import dropna
import pandas_ta as ta
from finta import TA

In [2]:
api_key = os.environ.get("api_key")
api_secret = os.environ.get("api_secret")

In [3]:
client = Client(api_key, api_secret)

In [4]:
# BTC Minute Data from past 50 days
historical_df = client.get_historical_klines('BTCUSDC', Client.KLINE_INTERVAL_1DAY, '1 Jan 2016')

In [5]:
hist_df = pd.DataFrame(historical_df)

In [6]:
hist_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,1544832000000,3200.00000000,3312.32000000,3000.00000000,3183.47000000,283.21385900,1544918399999,897046.63404699,1050,141.63907300,448709.23892821,0
1,1544918400000,3183.47000000,3257.31000000,3178.54000000,3199.27000000,423.04852900,1545004799999,1361883.39931020,1506,213.70001400,689502.92414080,0
2,1545004800000,3192.69000000,3587.35000000,3187.25000000,3494.65000000,1163.94029100,1545091199999,3968237.61401306,5062,438.24728700,1484453.16743536,0
3,1545091200000,3507.62000000,3683.35000000,3428.34000000,3670.11000000,1251.13962700,1545177599999,4409438.80579916,4707,739.77013000,2612032.94942774,0
4,1545177600000,3673.18000000,3920.00000000,3618.53000000,3676.32000000,2655.24282700,1545263999999,9978823.16211568,11297,1288.75405900,4849888.20273553,0


In [7]:
hist_df.columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 
                    'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore']

In [8]:
hist_df.tail()

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,TB Base Volume,TB Quote Volume,Ignore
1026,1633478400000,51502.83000000,55794.07000000,50408.96000000,55330.92000000,2133.37770100,1633564799999,113716193.94630434,64808,1038.53450800,55384143.31644654,0
1027,1633564800000,55332.46000000,55352.49000000,53392.38000000,53767.81000000,1434.18373000,1633651199999,78000326.33101130,49241,695.69303000,37858184.83960230,0
1028,1633651200000,53788.03000000,56150.90000000,53627.77000000,53918.86000000,1285.58196500,1633737599999,70304244.52617723,41679,612.56169500,33514670.05411793,0
1029,1633737600000,53953.98000000,55500.00000000,53684.22000000,54967.95000000,1105.48293000,1633823999999,60575064.10968080,29103,522.53372000,28631082.69150740,0
1030,1633824000000,54964.06000000,56103.79000000,54096.07000000,55457.50000000,932.93632000,1633910399999,51520520.26479300,25865,448.09196000,24752390.04585690,0


In [9]:
# hist_df['Open Time'] = pd.to_datetime(hist_df['Open Time']/1000, unit='s')
# hist_df['Close Time'] = pd.to_datetime(hist_df['Close Time']/1000, unit='s')

In [10]:
numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume', 'TB Base Volume', 'TB Quote Volume']

In [11]:
hist_df[numeric_columns] = hist_df[numeric_columns].apply(pd.to_numeric, axis=1)

In [12]:
hist_df.tail()

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,TB Base Volume,TB Quote Volume,Ignore
1026,1633478400000,51502.83,55794.07,50408.96,55330.92,2133.377701,1633564799999,1.137162e+08,64808,1038.534508,5.538414e+07,0
1027,1633564800000,55332.46,55352.49,53392.38,53767.81,1434.183730,1633651199999,7.800033e+07,49241,695.693030,3.785818e+07,0
1028,1633651200000,53788.03,56150.90,53627.77,53918.86,1285.581965,1633737599999,7.030424e+07,41679,612.561695,3.351467e+07,0
1029,1633737600000,53953.98,55500.00,53684.22,54967.95,1105.482930,1633823999999,6.057506e+07,29103,522.533720,2.863108e+07,0
1030,1633824000000,54964.06,56103.79,54096.07,55457.50,932.936320,1633910399999,5.152052e+07,25865,448.091960,2.475239e+07,0


In [13]:
hist_df.dtypes


Open Time               int64
Open                  float64
High                  float64
Low                   float64
Close                 float64
Volume                float64
Close Time              int64
Quote Asset Volume    float64
Number of Trades        int64
TB Base Volume        float64
TB Quote Volume       float64
Ignore                 object
dtype: object

In [14]:
hist_df.describe()


,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,TB Base Volume,TB Quote Volume
count,1.031000e+03,1031.000000,1031.000000,1031.000000,1031.000000,1031.000000,1.031000e+03,1.031000e+03,1031.000000,1031.000000,1.031000e+03
mean,1.589328e+12,18864.379564,19462.447856,18215.203094,18914.704918,2121.781135,1.589414e+12,4.520762e+07,30003.298739,1021.663237,2.199285e+07
std,2.572719e+10,16951.164364,17534.913047,16305.932269,16981.923682,1387.085616,2.572719e+10,5.528707e+07,23900.458711,684.050020,2.740574e+07
min,1.544832e+12,3183.470000,3257.310000,3000.000000,3183.470000,283.213859,1.544918e+12,8.970466e+05,1050.000000,122.332602,4.487092e+05
25%,1.567080e+12,7877.490000,8103.625000,7581.035000,7885.735000,1201.107428,1.567166e+12,1.086185e+07,12283.000000,556.614359,5.274732e+06
50%,1.589328e+12,10154.840000,10371.200000,9830.000000,10158.380000,1818.059387,1.589414e+12,2.047388e+07,22565.000000,876.085216,9.741190e+06
75%,1.611576e+12,32920.695000,34093.180000,31255.000000,33013.815000,2613.518779,1.611662e+12,6.125853e+07,43509.000000,1277.251404,2.933688e+07
max,1.633824e+12,63583.850000,64882.470000,62032.950000,63554.560000,13553.429477,1.633910e+12,5.128652e+08,226279.000000,6347.727851,2.410801e+08


In [ ]:
hist_df.info()

In [17]:
hist_df['Open Time'] = pd.to_datetime(hist_df['Open Time']/1000, unit='s')

hist_df['Close Time'] = pd.to_datetime(hist_df['Close Time']/1000, unit='s')

TypeError: cannot perform __truediv__ with this index type: DatetimeArray

In [18]:
# Just get the Open Time, Open, High, Low, Close, Volume
btc_ohlcv_daily = hist_df.iloc[:,0:6]

In [19]:
# CSV SAVER TOOL For Saving Bitcoin data to CSVs to use for past 50 days Minute data for Deep Learning Model
btc_ohlcv_daily.to_csv('btc_daily_ohlcv_2016to2021.csv', index=False)